In [3]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from awq import AutoAWQForCausalLM

import sys

sys.path.append("/home/hyohyeongjang/.conda/envs/kivi/lib/python3.10/site-packages/")
from models.llama_kivi import LlamaForCausalLM_KIVI

import torch, importlib, time, argparse, os, sys
from transformers import (
    HqqConfig,
    QuantoConfig,
    GPTQConfig,
    BitsAndBytesConfig,
    AwqConfig,
)

utils_path = os.path.abspath("/home/hyohyeongjang/myUtils/")
sys.path.append(utils_path)
import deviceUtils, IOUtils, checkUtils, modelUtils, parallelUtils, timeUtils

2222222222222222 Library(kind=FRAGMENT, ns=debugprims, dispatch_key=)>
2222222222222222 Library(kind=FRAGMENT, ns=debugprims, dispatch_key=)>
WARNING 07-24 20:03:39 _custom_ops.py:14] Failed to import from vllm._C with ImportError('/home/hyohyeongjang/.conda/envs/kivi/lib/python3.10/site-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c104impl3cow11cow_deleterEPv')
2222222222222222 quanto_ext::dqmm


RuntimeError: Failed to import transformers.generation.configuration_utils because of the following error (look up to see its traceback):
'str' object has no attribute 'impl'

In [ ]:
# #lower the version. Make it back!!!
# !conda install pytorch=2.1
# !conda install --upgrade pytorch
# !cd vLLM/EETQ
# !pip install vLLM/EETQ/

In [ ]:
# within main
prompts = ["The biggest challenge we face is"]
checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"


class MyQuantization:
    def __init__(self, tokenizer):
        self.hqq_config = HqqConfig(
            nbits=4, group_size=64, quant_zero=False, quant_scale=False, axis=0
        )  # replace all linear layers(e.g. attention query, key, value..etc)

        self.quanto_config = QuantoConfig(weights="int8")

        # 8 bits로 바꿔서 다시 해보기
        self.gptq_config = GPTQConfig(bits=4, dataset="c4", tokenizer=tokenizer)

        self.bitsandbytesconfig = BitsAndBytesConfig(load_in_8bit=True)

        self.awqconfig = AwqConfig(bits=8, fuse_max_seq_len=512, do_fuse=True)


t = AutoTokenizer.from_pretrained(checkpoint)
q = MyQuantization(t)
deviceUtils.DeviceUtils.gpu_usage()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['|Device 0| Mem Free: 70492.31MB / 81920.00MB(used: 11427.69MB, 13.9%)']

In [ ]:
model, tokenizer = modelUtils.ModelUtils.load_model_to_device(
    model_checkpoint=checkpoint,
    model_loading_class=AutoModelForCausalLM,
    tokenizer_loading_class=AutoTokenizer,
    # quantization_config=None,
    # quantization_config=q.hqq_config,
    # quantization_config=q.quanto_config,
    # quantization_config=q.gptq_config,
    # quantization_config=q.bitsandbytesconfig,
    # quantization_config=q.bitsandbytesconfig,
)

tokenizer.pad_token_id = tokenizer.eos_token_id
deviceUtils.DeviceUtils.gpu_usage()

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attentio

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Total Parameters:  8,030,261,248


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['|Device 0| Mem Free: 38972.31MB / 81920.00MB(used: 42947.69MB, 52.4%)']

In [ ]:
@timeUtils.TimeUtils.consumedTime_decorator
def main(args, batch):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = args.tokenizer(batch, return_tensors="pt", padding=True).to(device)
    outputs = args.model.generate(
        **inputs,
        max_length=512,
        min_length=512,
        do_sample=False,
        pad_token_id=args.tokenizer.eos_token_id
    )

    return args.tokenizer.batch_decode(outputs)

In [ ]:
types = "jupyter_inline"
if __name__ == "__main__":

    if types == "argumentparser":
        parser = argparse.ArgumentParser()
        parser.add_argument("--model", default=None, type=str, required=True)
        parser.add_argument("--tokenizer", default=None, type=str, required=False)
        parser.add_argument("--prompt", default=None, type=str, required=False)
        args = parser.parse_args()

    if types == "jupyter_inline":
        model_checkpoint = ""
        tokenizer_checkpoint = ""
        prompt = ""
        args = argparse.Namespace(model=model, tokenizer=tokenizer, prompt=prompt)

    if types in ["argumentparser", "jupyter_inline"]:
        out = main((args, prompts))
        print(deviceUtils.DeviceUtils.gpu_usage())

/home/hyohyeongjang/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/hyohyeongjang/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


72.0 seconds consumed
['|Device 0| Mem Free: 70492.31MB / 81920.00MB(used: 11427.69MB, 13.9%)']


In [ ]:
print(len(tokenizer.encode(out[0])))

513


In [ ]:
deviceUtils.DeviceUtils.gpu_usage()

['|Device 0| Mem Free: 70492.31MB / 81920.00MB(used: 11427.69MB, 13.9%)']

In [ ]:
out[0]

'<|begin_of_text|>The biggest challenge we face is the lack of awareness about the importance of mental health. Many people still view mental health as a taboo topic, and this can make it difficult for individuals to seek help when they need it.\nAnother challenge is the stigma surrounding mental health. Many people may feel ashamed or embarrassed to admit that they are struggling with their mental health, and this can prevent them from seeking the help they need.\nAdditionally, there is a shortage of mental health professionals and resources in many areas, which can make it difficult for people to access the help they need.\nOverall, the biggest challenge we face is the lack of awareness and understanding about mental health, and the stigma surrounding it.\n\nThe biggest challenge we face is the lack of awareness about the importance of mental health. Many people still view mental health as a taboo topic, and this can make it difficult for individuals to seek help when they need it.\n